In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import calendar
import matplotlib.pyplot as plt
import requests
from io import StringIO

In [2]:
# ==================================================
# Function to clean data 
# Objective: Split the text that comes with the '||' format to separate the categories for each column
# Parameters:
#     df = DataFrame to use
#     indx = Main column to merge with the original DataFrame
#     col = Name of the colum to clean within the original DataFrame
#     txtRemove = Category we need to remove after cleansing process
#     pivotTable = Pivot table to deliver only one single index to merge with the original Dataset (if needed)
# ==================================================

def splitColumnsDF(df, indx, col, txtRemove = "", pivotTable = False):
    print("======================================")
    # Identify DataFrame to clean all columns    
    tempDF = df[[indx,col,'year']]
    
    # Filter to get the desire years of analysis    
    #tempDF = tempDF.loc[(tempDF["year"] >= 2014) & (tempDF["year"] <= 2017)]
    print(f"Starting analysis for {col}...")
    
    
    print(f"Removing empty values...")
    # Removing empty values
    tempDF.dropna(axis=1,how="all")
    tempDF = tempDF[tempDF[col].notna()]
    
    
    
    # Debuggin
    #display(tempDF.head())
    
    # Create temporary dataframe to split rows by |
    internalTempDf = pd.DataFrame(tempDF[col].str.split("|",expand = True))
    # reset index to allow duplicates in 'indx'. The split will cause duplicate rows for different values
    internalTempDf = internalTempDf.reset_index()
    
    print("Splitting dataframe...")
    # Debugging   
    #display(internalTempDf.head())
    #display(internalTempDf.shape)
    
    # Creatting a transpose DataFrame to allow multiple rows for each splitted column
    internalTempDf = internalTempDf.melt(id_vars = ["index"], var_name = "Total", value_name="Value")
    # Set 1 for each row to sum up when aggregating data
    internalTempDf["Total"] = 1
    # Drop na values (for all the empty values. Reduce size)
    internalTempDf = internalTempDf.dropna(how="any")
    # Replace all the [#::] pattern to clean format
    internalTempDf["Value"] = internalTempDf["Value"].str.replace("[0-9]::?","", regex=True)
      
    # Debugging
    #display(internalTempDf.head(100))
    print("Grouping values...")
    # Group categories in the DataFrame to avoiod duplicates. Sum up the Total count perr category
    internalTempDf = internalTempDf.groupby(by=["index","Value"], as_index=False).sum()
    
    # Remove empty values (if needed) or 'Uknown' categoires in the data by accessing the index
    condicionRemove = internalTempDf.loc[(internalTempDf["Value"] == "")].index
    # Drop the index rows from the dataframe
    internalTempDf.drop(condicionRemove, inplace=True)
    # Adjunst the index to match the original indx
    internalTempDf.set_index("index",inplace=True)
    
    # Debugging
    #display(internalTempDf.head(100))
    print(f"Mergin {indx}...")
    # Merge the original dataframe to acces the indx column
    internalTempDf = internalTempDf.merge(tempDF[indx], how="left", left_index=True, right_index=True)
    # Debugging
    #display(internalTempDf.head(100))
    #display(t["incident_id"].count())
    print("Completed!")
    # Return the cleaned dataframe 
    return internalTempDf

In [10]:
# Import Data gun file 
# Load the entire file
urlGunViolence = "../gun-violence-data_01-2013_03-2018.csv"
# Sample data for testing purposes
#urlGunViolence = "Resources/gun_violence_data_small.csv"

# load the data (reading 50,000 rows)
gunViolenceDf = pd.read_csv(urlGunViolence,nrows=80000)

# Adding useful rows for plotting
gunViolenceDf['date'] = pd.to_datetime(gunViolenceDf['date'])
gunViolenceDf['year'] = gunViolenceDf['date'].dt.year
gunViolenceDf['month'] = gunViolenceDf['date'].dt.month
gunViolenceDf['monthday'] = gunViolenceDf['date'].dt.day
gunViolenceDf['weekday'] = gunViolenceDf['date'].dt.weekday

In [11]:
# Starts cleaning data
#display(gunViolenceDf.columns)
#display(gunViolenceDf.head())

#gunViolenceDf.head()
# Cleaning categories
gunStolenDF = splitColumnsDF(gunViolenceDf,"incident_id","gun_stolen", "Unknown", True)
gunTypeDF = splitColumnsDF(gunViolenceDf,"incident_id","gun_type", "Unknown", False)
######characteristicsDF = splitColumnsDF(gunViolenceDf,"incident_id","incident_characteristics", "Unknown", True)
ageDF = splitColumnsDF(gunViolenceDf,"incident_id","participant_age", "Unknown", False)
ageGroupDF = splitColumnsDF(gunViolenceDf,"incident_id","participant_age_group", "Unknown", True)
genderDF = splitColumnsDF(gunViolenceDf,"incident_id","participant_gender", "Unknown", True)
relationshipDF = splitColumnsDF(gunViolenceDf,"incident_id","participant_relationship", "Unknown", False)
statusDF = splitColumnsDF(gunViolenceDf,"incident_id","participant_status", "Unknown", False)
typeDF = splitColumnsDF(gunViolenceDf,"incident_id","participant_type", "Unknown", True)

# Function to clean all the columns within the DataFrame

# remove unwanted columns from original DataFrame


Starting analysis for gun_stolen...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Starting analysis for gun_type...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Starting analysis for participant_age...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Starting analysis for participant_age_group...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Starting analysis for participant_gender...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Starting analysis for participant_relationship...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Starting analysis for participant_status...
Removing empty values...
Splitting dataframe...
Grouping values...
Mergin incident_id...
Completed!
Star

In [12]:
display(gunStolenDF.head())
display(gunTypeDF.head())
display(ageDF.head())
display(ageGroupDF.head())
display(genderDF.head())
display(relationshipDF.head())
display(statusDF.head())
display(typeDF.head())

,Value,Total,incident_id
2,Unknown,2,478855
4,Unknown,2,478959
6,Unknown,2,479363
10,Unknown,1,491674
11,Unknown,1,479413


,Value,Total,incident_id
2,Unknown,2,478855
4,Handgun,2,478959
6,22 LR,1,479363
6,223 Rem [AR-15],1,479363
10,Unknown,1,491674


,Value,Total,incident_id
0,20,1,461105
1,20,1,460726
2,25,1,478855
2,31,1,478855
2,33,2,478855


,Value,Total,incident_id
0,Adult 18+,5,461105
1,Adult 18+,4,460726
2,Adult 18+,5,478855
3,Adult 18+,4,478925
4,Adult 18+,3,478959


,Value,Total,incident_id
0,Female,1,461105
0,Male,3,461105
1,Male,1,460726
2,Male,5,478855
3,Female,1,478925


,Value,Total,incident_id
4,Family,1,478959
6,Family,1,479363
16,Drive by - Random victims,3,479580
36,Aquaintance,1,482856
44,Gang vs Gang,2,483835


,Value,Total,incident_id
0,Arrested,1,461105
0,Injured,4,461105
1,Injured,3,460726
1,Killed,1,460726
2,Injured,2,478855


,Value,Total,incident_id
0,Subject-Suspect,1,461105
0,Victim,4,461105
1,Subject-Suspect,1,460726
1,Victim,4,460726
2,Subject-Suspect,2,478855


,Value,Total,incident_id
0,Arrested,1,461105
0,Injured,4,461105
